In [1]:
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/carvana-image-masking-challenge/train_hq.zip
/kaggle/input/carvana-image-masking-challenge/train_masks.zip
/kaggle/input/carvana-image-masking-challenge/train.zip
/kaggle/input/carvana-image-masking-challenge/29bb3ece3180_11.jpg
/kaggle/input/carvana-image-masking-challenge/train_masks.csv.zip
/kaggle/input/carvana-image-masking-challenge/test_hq.zip
/kaggle/input/carvana-image-masking-challenge/sample_submission.csv.zip
/kaggle/input/carvana-image-masking-challenge/metadata.csv.zip
/kaggle/input/carvana-image-masking-challenge/test.zip


In [2]:
import zipfile
from skimage.util import random_noise
import matplotlib.pylab as plt

In [3]:
DATA_PATH = '../input/carvana-image-masking-challenge/'
# Will unzip the files so that you can see them..
with zipfile.ZipFile(DATA_PATH + "train_masks.csv.zip","r") as z:
    z.extractall(".")

In [4]:
# Will unzip the files so that you can see them..
with zipfile.ZipFile(DATA_PATH + "train.zip","r") as z:
    z.extractall(".")

In [5]:
# Will unzip the files so that you can see them..
with zipfile.ZipFile(DATA_PATH + "train_masks.zip","r") as z:
    z.extractall(".")

In [6]:
import os 
from glob import glob

INPUT_PATH = '.'
DATA_PATH = INPUT_PATH
TRAIN_DATA = os.path.join(DATA_PATH, "train")
TRAIN_MASKS_DATA = os.path.join(DATA_PATH, "train_masks")

In [7]:
train_files = np.array(glob(os.path.join(TRAIN_DATA, "*.jpg")))

In [8]:
train_ids_ = np.array([s[len(TRAIN_DATA)+1:-6] for s in train_files])

In [9]:
train_ids_ = np.unique(train_ids_)

In [10]:
len(train_ids_)

318

In [11]:
train_ids = train_ids_[:250]
test_ids = train_ids_[250:]

In [12]:
# image_size = (864//2,576//2)
image_size = (128,128)

In [ ]:
def random_flip(img, mask, u=0.5):
    if np.random.random() < u:
        img = flip_axis(img, 1)
        mask = flip_axis(mask, 1)
    return img, mask

def rotate(x, theta, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest', cval=0.):
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                                [np.sin(theta), np.cos(theta), 0],
                                [0, 0, 1]])
    h, w = x.shape[row_axis], x.shape[col_axis]
    transform_matrix = img_gen.transform_matrix_offset_center(rotation_matrix, h, w)
    x = img_gen.apply_transform(x, transform_matrix, channel_axis, fill_mode, cval)
    return x

def random_rotate(img, mask, rotate_limit=(-20, 20), u=0.5):
    if np.random.random() < u:
        theta = np.pi / 180 * np.random.uniform(rotate_limit[0], rotate_limit[1])
        img = ndimage.rotate(img, theta, reshape=False)
        mask = ndimage.rotate(mask, theta, reshape=False)

#         img = rotate(img, theta)
#         mask = rotate(mask, theta)
    return img, mask

def random_shift(img, mask, w_limit=(-0.1, 0.1), h_limit=(-0.1, 0.1), u=0.5):
    if np.random.random() < u:
        wshift = np.random.uniform(w_limit[0], w_limit[1])
        hshift = np.random.uniform(h_limit[0], h_limit[1])
                
        img = ndimage.shift(img, (wshift, hshift, 0))
        mask = ndimage.shift(mask, (wshift, hshift))
#         img = shift(img, wshift, hshift)
#         mask = shift(mask, wshift, hshift)
    return img, mask

def shift(x, wshift, hshift, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest', cval=0.):
    h, w = x.shape[row_axis], x.shape[col_axis]
    tx = hshift * h
    ty = wshift * w
    translation_matrix = np.array([[1, 0, tx],
                                   [0, 1, ty],
                                   [0, 0, 1]])
    transform_matrix = translation_matrix  # no need to do offset
    x = img_gen.apply_transform(x, transform_matrix, channel_axis, fill_mode, cval)
    return x

def zoom(x, zx, zy, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest', cval=0.):
    zoom_matrix = np.array([[zx, 0, 0],
                            [0, zy, 0],
                            [0, 0, 1]])
    h, w = x.shape[row_axis], x.shape[col_axis]
    transform_matrix = img_gen.transform_matrix_offset_center(zoom_matrix, h, w)
    x = img_gen.apply_transform(x, transform_matrix, channel_axis, fill_mode, cval)
    return x

def random_zoom(img, mask, zoom_range=(0.8, 1), u=0.5):
    if np.random.random() < u:
        zx, zy = np.random.uniform(zoom_range[0], zoom_range[1], 2)
        img = zoom(img, zx, zy)
        mask = zoom(mask, zx, zy)
    return img, mask

def shear(x, shear, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest', cval=0.):
    shear_matrix = np.array([[1, -np.sin(shear), 0],
                             [0, np.cos(shear), 0],
                             [0, 0, 1]])
    h, w = x.shape[row_axis], x.shape[col_axis]
    transform_matrix = img_gen.transform_matrix_offset_center(shear_matrix, h, w)
    x = img_gen.apply_transform(x, transform_matrix, channel_axis, fill_mode, cval)
    return x

def random_shear(img, mask, intensity_range=(-0.5, 0.5), u=0.5):
    if np.random.random() < u:
        sh = np.random.uniform(-intensity_range[0], intensity_range[1])
        img = shear(img, sh)
        mask = shear(mask, sh)
    return img, mask

def random_channel_shift(x, limit, channel_axis=2):
    x = np.rollaxis(x, channel_axis, 0)
    min_x, max_x = np.min(x), np.max(x)
    channel_images = [np.clip(x_ch + np.random.uniform(-limit, limit), min_x, max_x) for x_ch in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_axis + 1)
    return x

def random_gray(img, u=0.5):
    if np.random.random() < u:
        coef = np.array([[[0.114, 0.587, 0.299]]])  # rgb to gray (YCbCr)
        gray = np.sum(img * coef, axis=2)
        img = np.dstack((gray, gray, gray))
    return img

def random_contrast(img, limit=(-0.3, 0.3), u=0.5):
    if np.random.random() < u:
        alpha = 1.0 + np.random.uniform(limit[0], limit[1])
        coef = np.array([[[0.114, 0.587, 0.299]]])  # rgb to gray (YCbCr)
        gray = img * coef
        gray = (3.0 * (1.0 - alpha) / gray.size) * np.sum(gray)
        img = alpha * img + gray
        img = np.clip(img, 0., 1.)
    return img

def random_brightness(img, limit=(-0.3, 0.3), u=0.5):
    if np.random.random() < u:
        alpha = 1.0 + np.random.uniform(limit[0], limit[1])
        img = alpha * img
        img = np.clip(img, 0., 1.)
    return img

def random_saturation(img, limit=(-0.3, 0.3), u=0.5):
    if np.random.random() < u:
        alpha = 1.0 + np.random.uniform(limit[0], limit[1])
        coef = np.array([[[0.114, 0.587, 0.299]]])
        gray = img * coef
        gray = np.sum(gray, axis=2, keepdims=True)
        img = alpha * img + (1. - alpha) * gray
        img = np.clip(img, 0., 1.)
    return img

def random_augmentation(img, mask):
    img = random_channel_shift(img, limit=0.05)
    img = random_brightness(img, (-0.5, 0.5), 0.5)
    img = random_contrast(img, (-0.5, 0.5), 0.5)
    img = random_saturation(img, (-0.5, 0.5), 0.5)
    img = random_gray(img, 0.2)
    img, mask = random_rotate(img, mask, (-20, 20), 0.5)
#     img, mask = random_shear(img, mask, (-0.3, 0.3), 0.2)
    img, mask = random_flip(img, mask, 0.3)
    img, mask = random_shift(img, mask, (-20, 20), (-20, 20), 0.3)
#     img, mask = random_zoom(img, mask, (0.8, 1), 0.3)
    return img, mask

In [13]:
def get_filename(image_id, image_type):
    check_dir = False
    if "Train" == image_type:
        ext = 'jpg'
        data_path = TRAIN_DATA
        suffix = ''
    elif "Train_mask" in image_type:
        ext = 'gif'
        data_path = TRAIN_MASKS_DATA
        suffix = '_mask'
    elif "Test" in image_type:
        ext = 'jpg'
        data_path = TEST_DATA
        suffix = ''
    else:
        raise Exception("Image type '%s' is not recognized" % image_type)

    if check_dir and not os.path.exists(data_path):
        os.makedirs(data_path)

    return os.path.join(data_path, "{}{}.{}".format(image_id, suffix, ext))

import cv2
from PIL import Image

def mean_shift(input_img):
    # Shape of original image
    originShape = input_img.shape
    # Converting image into array of dimension [nb of pixels in originImage, 3]
    # based on r g b intensities    
    flatImg=np.reshape(input_img, [-1, 3])


    # Estimate bandwidth for meanshift algorithm    
    bandwidth = estimate_bandwidth(flatImg, quantile=0.1, n_samples=50)    
    ms = MeanShift(bandwidth = bandwidth, bin_seeding=True)

    # Performing meanshift on flatImg    
    ms.fit(flatImg)

    # (r,g,b) vectors corresponding to the different clusters after meanshift    
    labels=ms.labels_
    
    # Remaining colors after meanshift    
    cluster_centers = ms.cluster_centers_    

    # Finding and diplaying the number of clusters    
    labels_unique = np.unique(labels)    
    n_clusters_ = len(labels_unique)
    
    return np.reshape(labels, (*input_img.shape[:2], 1))


def get_image_data(image_id, image_type, **kwargs):
    if 'mask' in image_type:
        img = _get_image_data_pil(image_id, image_type, **kwargs)
    else:
        img = _get_image_data_opencv(image_id, image_type, **kwargs)
    return img

def _get_image_data_opencv(image_id, image_type, **kwargs):
    fname = get_filename(image_id, image_type)
    img = cv2.imread(fname)
    assert img is not None, "Failed to read image : %s, %s" % (image_id, image_type)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if True:
        img = cv2.resize(img, image_size)
        # Add salt-and-pepper noise to the image.
#         img = random_noise(img, mode='s&p',amount=0.2)
    return img/np.max(img)


def _get_image_data_pil(image_id, image_type, return_exif_md=False, return_shape_only=False):
    fname = get_filename(image_id, image_type)
    try:
        img_pil = Image.open(fname)
    except Exception as e:
        assert False, "Failed to read image : %s, %s. Error message: %s" % (image_id, image_type, e)

    if return_shape_only:
        return img_pil.size[::-1] + (len(img_pil.getbands()),)
    if True:
#         img_pil = img_pil.resize((388, 388))
        img_pil = img_pil.resize(image_size)
    img = np.asarray(img_pil)
    assert isinstance(img, np.ndarray), "Open image is not an ndarray. Image id/type : %s, %s" % (image_id, image_type)
    if not return_exif_md:
        return img
    else:
        return img, img_pil._getexif()

In [14]:
from tensorflow import keras

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

import tensorflow as tf

import numpy as np
import pylab as plt

In [15]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(32,32,32), n_channels=1, 
                 shuffle=True, image_type='Train', mask_image_type='Train_mask'):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()
        self.image_type = image_type
        self.mask_image_type = mask_image_type
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        # cv2 recevrses height and width
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, frames, *dim, n_channels)
        # Initialization
        frames = 17  # 1-16
        batch_size = len(list_IDs_temp)
    
        X = np.empty((batch_size, frames - 1, *image_size, 3), dtype=np.float)
        y = np.empty((batch_size, frames - 1, *image_size), dtype=np.float)

        # Generate data
        for b, ID in enumerate(list_IDs_temp):
            for i in range(1, frames):
                if i < 10:
                    ext = '0' + str(i)
                else:
                    ext = str(i)
                img, mask = get_image_data(ID + ext, self.image_type), get_image_data(ID + ext, self.mask_image_type)
                img, mask = random_augmentation(img, mask)
                X[b,i - 1,:,:,:] = img
                y[b,i - 1,:,:] = mask
                
#                 X[b,i - 1,:,:,:] = get_image_data(ID + ext, 'Train')
#                 y[b,i - 1,:,:] = get_image_data(ID + ext, 'Train_mask')

        return X, y

In [16]:
# Parameters
params = {'dim': image_size,
          'batch_size': 1,
          'n_channels': 3,
          'shuffle': True}

# Generators
training_generator = DataGenerator(train_ids, **params)
validation_generator = DataGenerator(test_ids, **params)

In [17]:
def convLSTM(input_img):
    convLSTM1 = ConvLSTM2D(filters=128, kernel_size=(3, 3),
                   padding='same', data_format='channels_last', return_sequences=True, 
                           activation='relu')(input_img)
    
    convtd1 = TimeDistributed((Conv2D(128, kernel_size=3, padding = "same", activation = 'relu')))(convLSTM1)
    
    mptd1 = TimeDistributed(MaxPooling2D((2, 2), (2, 2)))(convtd1)
    
    convLSTM2 = ConvLSTM2D(filters=256, kernel_size=(3, 3),
                   padding='same', data_format='channels_last', return_sequences=True, 
                           activation='relu')(mptd1)
    
    convtd2 = TimeDistributed((Conv2D(256, kernel_size=3, padding = "same", activation = 'relu')))(convLSTM2)
    
    mptd2 = TimeDistributed(MaxPooling2D((2, 2), (2, 2)))(convtd2)
    
    convtd3 = TimeDistributed(Conv2D(512, kernel_size=3, activation = 'relu', 
                                       padding = 'same', kernel_initializer = 'he_normal') )(mptd2)
    convtd4 = TimeDistributed(Conv2D(512, kernel_size=3, activation = 'relu', 
                                       padding = 'same', kernel_initializer = 'he_normal') )(convtd3)
    
    
    upconvtd1 = TimeDistributed(Conv2DTranspose(256, kernel_size=2, strides=2, 
                                                padding='same',kernel_initializer = 'he_normal'))(convtd4)
    
    # concatenate convtd4
    merge1 = concatenate([convtd2, upconvtd1]) # 512 kernel
    
    convtd5 = TimeDistributed(Conv2D(256, kernel_size=3, activation = 'relu', 
                                       padding = 'same', kernel_initializer = 'he_normal') )(merge1)
    convtd6 = TimeDistributed(Conv2D(256, kernel_size=3, activation = 'relu', 
                                       padding = 'same', kernel_initializer = 'he_normal') )(convtd5)
    
    upconvtd2 = TimeDistributed(Conv2DTranspose(128, kernel_size=2, strides=2, 
                                        padding='same',kernel_initializer = 'he_normal'))(convtd6)
    
    # concatenate convtd2
    merge2 = concatenate([convtd1, upconvtd2])
    
    convtd7 = TimeDistributed(Conv2D(128, kernel_size=3, activation = 'relu', 
                                       padding = 'same', kernel_initializer = 'he_normal') )(merge2)
    convtd8 = TimeDistributed(Conv2D(128, kernel_size=3, activation = 'relu', 
                                       padding = 'same', kernel_initializer = 'he_normal') )(convtd7)
    
    convtd9 = TimeDistributed(Conv2D(1, kernel_size=3, activation = 'sigmoid', 
                                       padding = 'same', kernel_initializer = 'he_normal') )(convtd8)
    
    return convtd9

In [18]:
input_img = Input(shape = (16, *params['dim'], params['n_channels']))

def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=-1)
    denominator = tf.reduce_sum(y_true + y_pred, axis=-1)

    return (numerator + 1) / (denominator + 1)

autoencoder = Model(input_img, convLSTM(input_img))
autoencoder.compile(loss=keras.losses.binary_crossentropy, metrics=[dice_loss, keras.losses.binary_crossentropy], optimizer='adam')

In [19]:
autoencoder.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 16, 128, 128 0                                            
__________________________________________________________________________________________________
conv_lst_m2d (ConvLSTM2D)       (None, 16, 128, 128, 604160      input_1[0][0]                    
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 16, 128, 128, 147584      conv_lst_m2d[0][0]               
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 16, 64, 64, 1 0           time_distributed[0][0]           
______________________________________________________________________________________________

In [20]:
autoencoder.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=1, epochs=10)

Train for 250 steps, validate for 68 steps
Epoch 1/10
250/250 [==============================] - 226s 903ms/step - loss: 0.3512 - dice_loss: 0.3865 - binary_crossentropy: 0.3512 - val_loss: 0.1754 - val_dice_loss: 0.5815 - val_binary_crossentropy: 0.1754
Epoch 2/10
250/250 [==============================] - 210s 842ms/step - loss: 0.1331 - dice_loss: 0.7505 - binary_crossentropy: 0.1331 - val_loss: 0.0917 - val_dice_loss: 0.8582 - val_binary_crossentropy: 0.0917
Epoch 3/10
250/250 [==============================] - 207s 830ms/step - loss: 0.0655 - dice_loss: 0.8850 - binary_crossentropy: 0.0655 - val_loss: 0.0496 - val_dice_loss: 0.9092 - val_binary_crossentropy: 0.0496
Epoch 4/10
250/250 [==============================] - 207s 827ms/step - loss: 0.0459 - dice_loss: 0.9206 - binary_crossentropy: 0.0459 - val_loss: 0.0426 - val_dice_loss: 0.9316 - val_binary_crossentropy: 0.0426
Epoch 5/10
250/250 [==============================] - 207s 827ms/step - loss: 0.0323 - dice_loss: 0.9458 - bi

In [21]:
imgs = validation_generator.__getitem__(0)

training_images = imgs[0]
testing_images = imgs[1]

In [22]:
pred = autoencoder.predict(training_images)

In [27]:
plt.imshow(pred[0][15].reshape(128,128))

IndexError: index 16 is out of bounds for axis 0 with size 16